In [5]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from datetime import datetime, timedelta


# Gerar todas as datas no formato 'month-day'
start_date = datetime.strptime('01-01', '%m-%d')
end_date = datetime.strptime('01-01', '%m-%d')
date_generated = [start_date + timedelta(days=x) for x in range(0, (end_date - start_date).days + 1)]
datas = [date.strftime('%B-%-d').lower() for date in date_generated]

for data in datas:
    def verificar_acesso(url):
        try:
            response = requests.get(url)
            if response.status_code == 200:
                return response.content
            else:
                print(f"Erro ao acessar {url}: {response.status_code}")
                return None
        except requests.RequestException as e:
            print(f"Erro de requisição: {e}")
            return None

    def coletar_dados_perfil(conteudo):
        soup = BeautifulSoup(conteudo, 'html.parser')
        
        perfis = soup.find_all('div', class_='headerFiche')
        dados = []
        primeiros_nomes = []

        profissoes_lista = [
            'actor', 'actress', 'comedian', 'dancer', 'humorist', 'musician', 'composer', 'singer', 'music-hall artist', 
            'politician', 'historian', 'lawyer', 'journalist', 'reporter', 'tv/radio host', 'youtuber', 
            'athlete', 'mountaineer', 'chess player', 'bridge player', 'author', 'novelist', 'writer', 
            'painter', 'sculptor', 'cartoonist', 'photographer', 'spiritualist', 'astrologer', 'occultist', 
            'scientist', 'inventor', 'architect', 'chronicler', 'commentator', 'world event'
        ]

        us_states = {
            'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 
            'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 
            'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'
        }

        for perfil in perfis:
            try:
                a_tag = perfil.find('a')
                if a_tag and a_tag.contents and isinstance(a_tag.contents[0], str):
                    nome = a_tag.contents[0].strip()
                    if "Display" in nome:
                        nome = nome.split("Display")[0].strip()
                    primeiros_nomes.append(nome)
                else:
                    nome = 'N/A'
            
                fiche = perfil.find_next('table', class_='fiche')
                if fiche:
                    # Capturando o signo (Sun)
                    sun_element = fiche.find('td', string=re.compile('Sun:'))
                    sun = sun_element.find_next_sibling('td').get_text(strip=True).split()[-1] if sun_element else 'N/A'

                    # Capturando o ascendente (AS)
                    ascendant_element = fiche.find('td', string=re.compile('AS:'))
                    ascendant = ascendant_element.find_next_sibling('td').get_text(strip=True).split()[-1] if ascendant_element else 'N/A'

                    # Capturando a lua (Moon)
                    moon_element = fiche.find('td', string=re.compile('Moon:'))
                    moon = moon_element.find_next_sibling('td').get_text(strip=True).split()[-1] if moon_element else 'N/A'

                    # Capturando o dia da semana, cidade e país de nascimento
                    born_element = fiche.find('td', string=re.compile('Born:|Event:'))
                    event_type = 'Born' if born_element and 'Born:' in born_element.get_text() else 'Event'
                    born_date = born_element.find_next_sibling('td').get_text(strip=True) if born_element else 'N/A'
                    born_in_element = fiche.find('td', string=re.compile('In:'))
                    born_in = born_in_element.find_next_sibling('td').get_text(strip=True) if born_in_element else 'N/A'
                    city, country = 'N/A', 'N/A'
                    if born_in != 'N/A':
                        if "(" in born_in and ")" in born_in:
                            city_country = born_in.split('(')
                            city = city_country[0].strip()
                            country = city_country[1].replace(')', '').strip()
                        else:
                            city_country = born_in.rsplit(' ', 1)
                            city = city_country[0].strip()
                            country = city_country[1].strip() if len(city_country) > 1 else 'N/A'
                    
                    # Corrigindo valores de país
                    if re.match(r'\d+', country):
                        country = 'France'
                    if country in us_states:
                        country = 'United States'
                    if ',' in country:
                        country = country.split(',')[-1].strip()

                    # Capturando os dominantes
                    dominants_element = fiche.find('td', string=re.compile('Dominants:'))
                    dominants = dominants_element.find_next_sibling('td').get_text(" ", strip=True) if dominants_element else 'N/A'

                    # Capturando a numerologia
                    numerology_element = fiche.find('td', string=re.compile('Numerology:'))
                    numerology = numerology_element.find_next_sibling('td').get_text(strip=True) if numerology_element else 'N/A'
                    
                    # Capturando casas e elementos
                    if dominants != 'N/A':
                        dominants_split = dominants.split('/')
                        houses = re.findall(r'\d+', dominants_split[0]) if len(dominants_split) > 0 else ['N/A']
                        elements = dominants_split[1].strip() if len(dominants_split) > 1 else 'N/A'
                        cardinal = dominants_split[2].strip() if len(dominants_split) > 2 else 'N/A'
                    else:
                        houses, elements, cardinal = ['N/A'], 'N/A', 'N/A'
                else:
                    sun, ascendant, moon, born_date, city, country, dominants, numerology, houses, elements, cardinal = 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', ['N/A'], 'N/A', 'N/A'

            except AttributeError:
                sun, ascendant, moon, born_date, city, country, dominants, numerology, houses, elements, cardinal = 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', ['N/A'], 'N/A', 'N/A'
            
            try:
                if event_type == 'Born':
                    biografia_div = perfil.find_next('div', style="font-weight: bold; margin: 8px auto 3px").find_next('p')
                    if biografia_div:
                        biografia_texto = biografia_div.get_text().lower()
                        profissao = next((prof for prof in profissoes_lista if prof in biografia_texto), 'N/A')
                    else:
                        profissao = 'N/A'
                else:
                    profissao = 'world event'
            except AttributeError:
                profissao = 'N/A'

            dia, mes, ano, dia_semana = 'N/A', 'N/A', 'N/A', 'N/A'
            if born_date != 'N/A':
                try:
                    parts = born_date.split(',')
                    parts2 = data.split('-')
                    dia_semana = parts[0].strip()
                    dia, mes, ano = parts2[1], parts2[0].capitalize(), parts[2].strip()
                except ValueError:
                    dia_semana, dia, mes, ano = 'N/A', 'N/A', 'N/A', 'N/A'

            dados.append({
                'Nome': nome,
                'Signo': sun,
                'Ascendente': ascendant,
                'Lua': moon,
                'Profissão': profissao,
                'Dia da Semana': dia_semana,
                'Dia': dia,
                'Mês': mes,
                'Ano': ano,
                'Cidade': city,
                'País': country,
                'Dominantes': dominants,
                'Casas': ', '.join(houses),
                'Elementos': elements,
                'Cardinal': cardinal,
                'Numerologia': numerology
            })

            print(f"Nome: {nome}, Signo: {sun}, Ascendente: {ascendant}, Lua: {moon}, Profissão: {profissao}, Dia da Semana: {dia_semana}, Dia: {dia}, Mês: {mes}, Ano: {ano}, Cidade: {city}, País: {country}, Dominantes: {dominants}, Casas: {houses}, Elementos: {elements}, Cardinal: {cardinal}, Numerologia: {numerology}")

        return dados, primeiros_nomes

    def main():
        base_url = f'https://www.astrotheme.com/birthdays/{data}/'
        page_number = 1
        all_dados = []
        visited_first_names = set()

        while True:
            url = f'{base_url}{page_number}.htm'
            conteudo = verificar_acesso(url)
            if conteudo:
                dados_pagina, primeiros_nomes = coletar_dados_perfil(conteudo)
                if not dados_pagina or (primeiros_nomes and primeiros_nomes[0] in visited_first_names):  # Se não houver perfis ou o primeiro nome se repetir, parar a execução
                    break
                visited_first_names.add(primeiros_nomes[0])
                all_dados.extend(dados_pagina)
                page_number += 1
            else:
                break

        # Salvar dados em um DataFrame pandas
        colunas = ['Nome', 'Signo', 'Ascendente', 'Lua', 'Profissão', 'Dia da Semana', 'Dia', 'Mês', 'Ano', 'Cidade', 'País', 'Dominantes', 'Casas', 'Elementos', 'Cardinal', 'Numerologia']
        df = pd.DataFrame(all_dados, columns=colunas)
        df.to_csv(f'celebridades_{data}.csv', index=False)
        print(f"Total de perfis coletados: {len(all_dados)}")

        return df

    if __name__ == '__main__':
        df_celebridades = main()


Nome: United Kingdom, Signo: Capricorn, Ascendente: Libra, Lua: Cancer, Profissão: world event, Dia da Semana: Thursday, Dia: 1, Mês: January, Ano: 1801, Cidade: London, País: United Kingdom, Dominantes: Cancer , Libra , Capricorn Moon , Sun , Uranus Houses 10 , 4 , 5 / Air , Water / Cardinal, Casas: ['10', '4', '5'], Elementos: Air , Water, Cardinal: Cardinal, Numerologia: Birth Path 3
Nome: Michel Onfray, Signo: Capricorn, Ascendente: Capricorn, Lua: Virgo, Profissão: author, Dia da Semana: Thursday, Dia: 1, Mês: January, Ano: 1959, Cidade: Argentan, País: France, Dominantes: Capricorn , Scorpio , Sagittarius Sun , Saturn , Neptune Houses 1 , 8 , 12 / Earth , Water / Cardinal, Casas: ['1', '8', '12'], Elementos: Earth , Water, Cardinal: Cardinal, Numerologia: Birth Path 8
Nome: Christine Lagarde, Signo: Capricorn, Ascendente: Taurus, Lua: Leo, Profissão: politician, Dia da Semana: Sunday, Dia: 1, Mês: January, Ano: 1956, Cidade: Paris 9e, País: France, Dominantes: Capricorn , Scorpio

In [9]:
import pandas as pd
import matplotlib.pyplot as plt

# Caminho do arquivo total.csv
file_path = '/Users/smx/Desktop/estatisticas/total.csv'

# Ler o arquivo CSV em um DataFrame
df = pd.read_csv(file_path)

# Verificar as primeiras linhas do DataFrame para entender a estrutura
print(df.head())

# Verificar quais colunas estão disponíveis
print(df.columns)

# Função para agrupar profissões similares
def agrupar_profissoes(profissao):
    if profissao in ['actor', 'actress', 'comedian']:
        return 'actor/comedian'
    elif profissao in ['author', 'writer', 'novelist']:
        return 'author/writer'
    elif profissao in ['singer', 'musician', 'composer']:
        return 'singer/musician'
    elif profissao == 'tv/radio host':
        return 'tv/radio host'
    elif profissao == 'youtuber':
        return 'youtuber'
    elif profissao == 'athlete':
        return 'athlete'
    elif profissao == 'mountaineer':
        return 'mountaineer'
    elif profissao == 'chess player':
        return 'chess player'
    elif profissao == 'bridge player':
        return 'bridge player'
    elif profissao in ['painter', 'sculptor', 'cartoonist', 'photographer']:
        return 'artist'
    elif profissao in ['spiritualist', 'astrologer', 'occultist']:
        return 'spiritualist'
    elif profissao == 'scientist':
        return 'scientist'
    elif profissao == 'inventor':
        return 'inventor'
    elif profissao == 'architect':
        return 'architect'
    elif profissao in ['chronicler', 'commentator']:
        return 'commentator'
    elif profissao == 'world event':
        return 'world event'
    elif profissao == 'politician':
        return 'politician'
    elif profissao == 'historian':
        return 'historian'
    elif profissao == 'lawyer':
        return 'lawyer'
    elif profissao == 'journalist':
        return 'journalist'
    else:
        return 'other'

# Aplicar a função de agrupamento de profissões
df['Profissão Agrupada'] = df['Profissão'].apply(agrupar_profissoes)

# Contar a frequência de cada profissão agrupada
profession_counts = df['Profissão Agrupada'].value_counts()

# Criar o histograma
plt.figure(figsize=(10, 6))
profession_counts.plot(kind='bar')
plt.xlabel('Profissões Agrupadas')
plt.ylabel

                Nome      Signo   Ascendente        Lua    Profissão  \
0     United Kingdom  Capricorn        Libra     Cancer  world event   
1      Michel Onfray  Capricorn    Capricorn      Virgo       author   
2  Christine Lagarde  Capricorn       Taurus        Leo   politician   
3    Michèle Mercier  Capricorn  Sagittarius     Taurus      actress   
4     J. D. Salinger  Capricorn          NaN  Capricorn       writer   

  Dia da Semana  Dia      Mês   Ano    Cidade            País  \
0      Thursday    1  January  1801    London  United Kingdom   
1      Thursday    1  January  1959  Argentan          France   
2        Sunday    1  January  1956  Paris 9e          France   
3        Sunday    1  January  1939      Nice          France   
4     Wednesday    1  January  1919  New York   United States   

                                          Dominantes     Casas      Elementos  \
0  Cancer , Libra , Capricorn Moon , Sun , Uranus...  10, 4, 5    Air , Water   
1  Capricorn ,

<function matplotlib.pyplot.ylabel(ylabel, fontdict=None, labelpad=None, *, loc=None, **kwargs)>

AttributeError: `np.Inf` was removed in the NumPy 2.0 release. Use `np.inf` instead.

<Figure size 1000x600 with 1 Axes>

In [7]:
pip install --upgrade matplotlib numpy

Defaulting to user installation because normal site-packages is not writeable
  Using cached matplotlib-3.9.0-cp39-cp39-macosx_11_0_arm64.whl.metadata (11 kB)
Using cached matplotlib-3.9.0-cp39-cp39-macosx_11_0_arm64.whl (7.8 MB)
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.3.4
    Uninstalling matplotlib-3.3.4:
      Successfully uninstalled matplotlib-3.3.4
Note: you may need to restart the kernel to use updated packages.


In [11]:
import pandas as pd

# Função para agrupar profissões similares
def agrupar_profissoes(profissao):
    if profissao in ['actor', 'actress', 'comedian']:
        return 'actor/comedian'
    elif profissao in ['author', 'writer', 'novelist']:
        return 'author/writer'
    elif profissao in ['singer', 'musician', 'composer']:
        return 'singer/musician'
    elif profissao == 'tv/radio host':
        return 'tv/radio host'
    elif profissao == 'youtuber':
        return 'youtuber'
    elif profissao == 'athlete':
        return 'athlete'
    elif profissao == 'mountaineer':
        return 'mountaineer'
    elif profissao in ['chess player', 'bridge player']:
        return 'game player'
    elif profissao in ['painter', 'sculptor', 'cartoonist', 'photographer']:
        return 'artist'
    elif profissao in ['spiritualist', 'astrologer', 'occultist']:
        return 'spiritualist'
    elif profissao == 'scientist':
        return 'scientist'
    elif profissao == 'inventor':
        return 'inventor'
    elif profissao == 'architect':
        return 'architect'
    elif profissao in ['chronicler', 'commentator']:
        return 'commentator'
    elif profissao == 'world event':
        return 'world event'
    elif profissao == 'politician':
        return 'politician'
    elif profissao == 'historian':
        return 'historian'
    elif profissao == 'lawyer':
        return 'lawyer'
    elif profissao == 'journalist':
        return 'journalist'
    else:
        return 'other'

# Caminho do arquivo total.csv
file_path = '/Users/smx/Desktop/estatisticas/total.csv'

# Ler o arquivo CSV em um DataFrame
df = pd.read_csv(file_path)

# Aplicar a função de agrupamento de profissões
df['Profissão Agrupada'] = df['Profissão'].apply(agrupar_profissoes)

# Função para calcular porcentagens por dia da semana
def calcular_porcentagens_dia_semana(df):
    dias_semana = df['Dia da Semana'].unique()
    coluna_interesse = 'Profissão Agrupada'
    porcentagens_dia_semana = {}

    for profissao in df[coluna_interesse].unique():
        porcentagens_dia_semana[profissao] = {}
        total_profissao = df[df[coluna_interesse] == profissao].shape[0]
        for dia in dias_semana:
            total_dia_profissao = df[(df[coluna_interesse] == profissao) & (df['Dia da Semana'] == dia)].shape[0]
            porcentagem = (total_dia_profissao / total_profissao) * 100
            porcentagens_dia_semana[profissao][dia] = (total_dia_profissao, porcentagem)
    
    return porcentagens_dia_semana

# Função para calcular porcentagens por signo
def calcular_porcentagens_signo(df):
    signos = df['Signo'].unique()
    coluna_interesse = 'Profissão Agrupada'
    porcentagens_signo = {}

    for profissao in df[coluna_interesse].unique():
        porcentagens_signo[profissao] = {}
        total_profissao = df[df[coluna_interesse] == profissao].shape[0]
        for signo in signos:
            total_signo_profissao = df[(df[coluna_interesse] == profissao) & (df['Signo'] == signo)].shape[0]
            porcentagem = (total_signo_profissao / total_profissao) * 100
            porcentagens_signo[profissao][signo] = (total_signo_profissao, porcentagem)
    
    return porcentagens_signo

# Coletar os dados
porcentagens_dia_semana = calcular_porcentagens_dia_semana(df)
porcentagens_signo = calcular_porcentagens_signo(df)



In [12]:
import matplotlib.pyplot as plt

# Function to plot percentages by day of the week
def plot_weekday_percentages(weekday_percentages):
    for profession, data in weekday_percentages.items():
        days = list(data.keys())
        values = [v[1] for v in data.values()]
        total_profession = sum([v[0] for v in data.values()])

        plt.figure(figsize=(10, 6))
        plt.bar(days, values)
        plt.xlabel('Days of the Week')
        plt.ylabel('Percentage')
        plt.title(f'Percentage of {profession} by Day of the Week (Total: {total_profession})')
        plt.xticks(rotation=0)
        plt.tight_layout()
        plt.show()

# Function to plot percentages by zodiac sign
def plot_sign_percentages(sign_percentages):
    for profession, data in sign_percentages.items():
        signs = zodiac_order  # Ensure the traditional order
        values = [data[sign][1] if sign in data else 0 for sign in signs]
        total_profession = sum([data[sign][0] if sign in data else 0 for sign in signs])

        plt.figure(figsize=(10, 6))
        plt.bar(signs, values)
        plt.xlabel('Zodiac Signs')
        plt.ylabel('Percentage')
        plt.title(f'Percentage of {profession} by Zodiac Sign (Total: {total_profession})')
        plt.xticks(rotation=0)
        plt.tight_layout()
        plt.show()

# Function to plot percentages by moon
def plot_moon_percentages(moon_percentages):
    for profession, data in moon_percentages.items():
        moons = zodiac_order  # Ensure the traditional order
        values = [data[moon][1] if moon in data else 0 for moon in moons]
        total_profession = sum([data[moon][0] if moon in data else 0 for moon in moons])

        plt.figure(figsize=(10, 6))
        plt.bar(moons, values)
        plt.xlabel('Moons')
        plt.ylabel('Percentage')
        plt.title(f'Percentage of {profession} by Moon (Total: {total_profession})')
        plt.xticks(rotation=0)
        plt.tight_layout()
        plt.show()

# Function to plot percentages by ascendant, excluding 'Unknown'
def plot_ascendant_percentages(ascendant_percentages):
    for profession, data in ascendant_percentages.items():
        ascendents = zodiac_order  # Ensure the traditional order
        values = [data[ascendant][1] if ascendant in data else 0 for ascendant in ascendents]
        total_profession = sum([data[ascendant][0] if ascendant in data else 0 for ascendant in ascendents])

        plt.figure(figsize=(10, 6))
        plt.bar(ascendents, values)
        plt.xlabel('Ascendants')
        plt.ylabel('Percentage')
        plt.title(f'Percentage of {profession} by Ascendant (Total: {total_profession}, Excluding Unknown Ascendants)')
        plt.xticks(rotation=0)
        plt.tight_layout()
        plt.show()

# Plot the data
plot_weekday_percentages(weekday_percentages)
plot_sign_percentages(sign_percentages)
plot_moon_percentages(moon_percentages)
plot_ascendant_percentages(ascendant_percentages)


NameError: name 'weekday_percentages' is not defined